In [1]:
#-*- coding:utf-8 -*-

import requests, os
import bs4
from bs4 import BeautifulSoup

import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from urllib import parse
import re
import lxml
import json
import sys

In [2]:
class Instagram_crawler:
    def __init__(self, tag, no_of_scroll):
        self.tag = tag
        self.tag_url = "www.instagram.com/explore/tags/" + self.tag
        self.tag_url = "http://" + parse.quote(self.tag_url)
        print('\n', self.tag_url, '\n')
        
        num = 1
        self.shortcodes = []
        print('start to get shortcode')
        while num <= no_of_scroll:
            # get html code
            soup = self.scroll_page(num)
            # get the list of shortcode
            self.shortcodes.extend(self.get_shortcode_list(soup))
            print()
            num+=1

        # delete dupicated data
        shortcodes = list(set(self.shortcodes))
        print('  total shortcodes :', len(self.shortcodes))
        
        print('\nstart to get data')
        saved_data = self.get_data(shortcodes)
        print('  saved data :', saved_data)
        
    # scroll url page num times and get html data
    def scroll_page(self, num):
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        option.add_argument('disable-gpu')

        browser = webdriver.Chrome('chromedriver', options=option)

        # browser = webdriver.Chrome()
        browser.get(self.tag_url)
        time.sleep(3)

        body = browser.find_element_by_tag_name("body")

        print(" page scroll start (", num, ' times)')

        while num:
            no_of_pagedowns = 8;
            num-=1
            while no_of_pagedowns:
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(1)
                no_of_pagedowns-=1

        print(" page scrolled")
        response = browser.page_source
        soup = BeautifulSoup(response, 'lxml')
        print(" read the HTML file")

        return soup

    # return the list of shortcode from html code
    def get_shortcode_list(self, soup):
        article = soup.find("article")
        href = article.find_all('a')

        shortcodes = []

        for h in href:
            shortcodes.append(h['href'])

        print(" get ", len(shortcodes) - 9, " shortcodes")

        return shortcodes[9:]


    # return the text from html code
    def get_text(self, page):
        script = page.find_all("script")[3].string
        script_split = re.split(':|,|\"|\'|\{|\}|\[|\]',script)  
        text_index = script_split.index('text') + 3
        text = script_split[text_index]  

        text = text.replace('\\n','')

        text = text.encode('utf-8', 'surrogatepass')
        text = text.decode('unicode_escape')

        return text  

    # return the user_id and number of likes from html code
    def get_id_likes(self, description):
        data = re.split('on Instagram:', description)

        data = re.split(",|\(|\)|@|Likes|Comments|-| ",data[0])
        while '' in data : data.remove('')

        likes = data[0]
        user_id = data[-1]

        return user_id, likes

    # return the text, user_id, number of likes from html code
    def get_labels(self, page):
        for tag in page.select('meta[property*=image]'):
            image_url = tag['content']

        for tag in page.select('meta[property*=og:description]'):
            description = tag['content']

        text = self.get_text(page)
        user_id, likes = self.get_id_likes(description)

        return text, user_id, likes

    # save image in page
    def save_image(self, page, shortcode):
        for tag in page.select('meta[property*=image]'):
            image_url = tag['content']

        pic_response = requests.get(image_url).content
        img_name = "./dataset/img/" + shortcode[3:-1] + ".jpg"

        with open(img_name, 'wb') as f:
            f.write(pic_response)
        print(" file saved: ", img_name)    

    def get_data(self, shortcodes):
        i = 1
        saved_data = 0
        file_name = "./dataset/label.json"

        for shortcode in shortcodes:
            print("[ ", i, ' ]')
            i = i + 1  
            label_data = []

            try:
                # get html code
                page_url = "https://www.instagram.com" + shortcode
                print(' ', page_url)
                response = requests.get(page_url).content
                page = BeautifulSoup(response, 'lxml')        

                # get label data
                text, user_id, likes = self.get_labels(page)

                # save label data 
                output = {"shortcode":shortcode, "label":self.tag, "likes":likes, "user_id":user_id, "text":text}
                label_data.append(output)

                if not os.path.isfile(file_name):
                    with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                        new_feed = json.dumps(label_data, ensure_ascii=False, indent=2)
                        f.write(new_feed)
                else:
                    with open(file_name, encoding='UTF-16') as feedsjson:
                        old_feeds = json.load(feedsjson)
                    old_feeds.append(label_data)
                    with open(file_name, mode='w', encoding='UTF-16', errors='ignore') as f:
                        new_feed = json.dumps(old_feeds, ensure_ascii=False, indent=2)
                        f.write(new_feed) 

                # save image files
                self.save_image(page, shortcode)

                print(" Okay\n")
                saved_data += 1

            except:
                print(" Unexpected error:", sys.exc_info()[0], sys.exc_info()[1])
                print(" Line ", sys.exc_info()[-1].tb_lineno)
                print(" Not okay\n")

        return saved_data

In [3]:
tag = input("\n** 원하는 태그를 입력하세요: \n>>> ")
no_of_scroll = input("\n** 스크롤 할 페이지 수를 입력하세요.\n 한 번의 스크롤 마다 약 45개의 데이터가 획득됩니다.: \n>>> ")


** 원하는 태그를 입력하세요: 
>>> 경복궁

** 스크롤 할 페이지 수를 입력하세요.
 한 번의 스크롤 마다 약 45개의 데이터가 획득됩니다.: 
>>> 10


In [4]:
ic = Instagram_crawler(tag, int(no_of_scroll))


 http://www.instagram.com/explore/tags/%EA%B2%BD%EB%B3%B5%EA%B6%81 

start to get shortcode
 page scroll start ( 1  times)
 page scrolled
 read the HTML file
 get  51  shortcodes

 page scroll start ( 2  times)
 page scrolled
 read the HTML file
 get  51  shortcodes

 page scroll start ( 3  times)
 page scrolled
 read the HTML file
 get  48  shortcodes

 page scroll start ( 4  times)
 page scrolled
 read the HTML file
 get  48  shortcodes

 page scroll start ( 5  times)
 page scrolled
 read the HTML file
 get  48  shortcodes

 page scroll start ( 6  times)
 page scrolled
 read the HTML file
 get  45  shortcodes

 page scroll start ( 7  times)
 page scrolled
 read the HTML file
 get  45  shortcodes

 page scroll start ( 8  times)
 page scrolled
 read the HTML file
 get  39  shortcodes

 page scroll start ( 9  times)
 page scrolled
 read the HTML file
 get  42  shortcodes

 page scroll start ( 10  times)
 page scrolled
 read the HTML file
 get  42  shortcodes

  total shortcodes : 459



 file saved:  ./dataset/img/경복궁_BqRZkO0nUfg.jpg
 Okay

[  68  ]
  https://www.instagram.com/p/BqP_3OBgUtW/
 file saved:  ./dataset/img/경복궁_BqP_3OBgUtW.jpg
 Okay

[  69  ]
  https://www.instagram.com/p/Bpzd4Cige0x/
 file saved:  ./dataset/img/경복궁_Bpzd4Cige0x.jpg
 Okay

[  70  ]
  https://www.instagram.com/p/BqRgmueBJFa/
 file saved:  ./dataset/img/경복궁_BqRgmueBJFa.jpg
 Okay

[  71  ]
  https://www.instagram.com/p/BqQ_sv0FkTk/
 file saved:  ./dataset/img/경복궁_BqQ_sv0FkTk.jpg
 Okay

[  72  ]
  https://www.instagram.com/p/BqQuhtig2Dr/
 file saved:  ./dataset/img/경복궁_BqQuhtig2Dr.jpg
 Okay

[  73  ]
  https://www.instagram.com/p/BqRIfvvjv6c/
 file saved:  ./dataset/img/경복궁_BqRIfvvjv6c.jpg
 Okay

[  74  ]
  https://www.instagram.com/p/BqRbo3hlKHc/
 file saved:  ./dataset/img/경복궁_BqRbo3hlKHc.jpg
 Okay

[  75  ]
  https://www.instagram.com/p/BqRzzJlHfOG/
 file saved:  ./dataset/img/경복궁_BqRzzJlHfOG.jpg
 Okay

[  76  ]
  https://www.instagram.com/p/BqM0V7FAQ-K/
 file saved:  ./dataset/img/경복궁_BqM0V

 file saved:  ./dataset/img/경복궁_BqREh5DA5hJ.jpg
 Okay

[  142  ]
  https://www.instagram.com/p/BqR1dcBFqoW/
 file saved:  ./dataset/img/경복궁_BqR1dcBFqoW.jpg
 Okay

[  143  ]
  https://www.instagram.com/p/BqRsyljHM-a/
 file saved:  ./dataset/img/경복궁_BqRsyljHM-a.jpg
 Okay

[  144  ]
  https://www.instagram.com/p/BqQVUJgnps_/
 file saved:  ./dataset/img/경복궁_BqQVUJgnps_.jpg
 Okay

[  145  ]
  https://www.instagram.com/p/BqRyVIiAyrH/
 file saved:  ./dataset/img/경복궁_BqRyVIiAyrH.jpg
 Okay

[  146  ]
  https://www.instagram.com/p/BqRdxRNlkdG/
 file saved:  ./dataset/img/경복궁_BqRdxRNlkdG.jpg
 Okay

[  147  ]
  https://www.instagram.com/p/BqR008EBfkB/
 file saved:  ./dataset/img/경복궁_BqR008EBfkB.jpg
 Okay

[  148  ]
  https://www.instagram.com/p/BqRgzvUglLk/
 file saved:  ./dataset/img/경복궁_BqRgzvUglLk.jpg
 Okay

[  149  ]
  https://www.instagram.com/p/BqQFjkiA9No/
 file saved:  ./dataset/img/경복궁_BqQFjkiA9No.jpg
 Okay

[  150  ]
  https://www.instagram.com/p/BqRzUJen9ca/
 file saved:  ./dataset/img/

 file saved:  ./dataset/img/경복궁_BqQ0G_WgXAo.jpg
 Okay

[  218  ]
  https://www.instagram.com/p/BqRwT3RBu5O/
 file saved:  ./dataset/img/경복궁_BqRwT3RBu5O.jpg
 Okay

[  219  ]
  https://www.instagram.com/p/BqP-D8ChTor/
 file saved:  ./dataset/img/경복궁_BqP-D8ChTor.jpg
 Okay

[  220  ]
  https://www.instagram.com/p/BqQwWBXBgG7/
 file saved:  ./dataset/img/경복궁_BqQwWBXBgG7.jpg
 Okay

[  221  ]
  https://www.instagram.com/p/BqRM95uAPoq/
 file saved:  ./dataset/img/경복궁_BqRM95uAPoq.jpg
 Okay

[  222  ]
  https://www.instagram.com/p/BnXZXW5FLNG/
 file saved:  ./dataset/img/경복궁_BnXZXW5FLNG.jpg
 Okay

[  223  ]
  https://www.instagram.com/p/BqRJJFhAQHW/
 file saved:  ./dataset/img/경복궁_BqRJJFhAQHW.jpg
 Okay

[  224  ]
  https://www.instagram.com/p/BqP_QntAyoX/
 file saved:  ./dataset/img/경복궁_BqP_QntAyoX.jpg
 Okay

[  225  ]
  https://www.instagram.com/p/BqRXoDRlzwK/
 file saved:  ./dataset/img/경복궁_BqRXoDRlzwK.jpg
 Okay

[  226  ]
  https://www.instagram.com/p/BqP-ZYYgye8/
 file saved:  ./dataset/img/

 file saved:  ./dataset/img/경복궁_BqP-tMsAhk6.jpg
 Okay

[  292  ]
  https://www.instagram.com/p/BqRdmf-gTz7/
 file saved:  ./dataset/img/경복궁_BqRdmf-gTz7.jpg
 Okay

[  293  ]
  https://www.instagram.com/p/BqRwWfFHrnd/
 file saved:  ./dataset/img/경복궁_BqRwWfFHrnd.jpg
 Okay

[  294  ]
  https://www.instagram.com/p/BqQ_8sAAFqm/
 file saved:  ./dataset/img/경복궁_BqQ_8sAAFqm.jpg
 Okay

[  295  ]
  https://www.instagram.com/p/BqRMqKDgfJj/
 file saved:  ./dataset/img/경복궁_BqRMqKDgfJj.jpg
 Okay

[  296  ]
  https://www.instagram.com/p/BqR03o5FC8H/
 file saved:  ./dataset/img/경복궁_BqR03o5FC8H.jpg
 Okay

[  297  ]
  https://www.instagram.com/p/BqRzIwkBD89/
 file saved:  ./dataset/img/경복궁_BqRzIwkBD89.jpg
 Okay

[  298  ]
  https://www.instagram.com/p/BqQmJp1lsmb/
 file saved:  ./dataset/img/경복궁_BqQmJp1lsmb.jpg
 Okay

[  299  ]
  https://www.instagram.com/p/BqRpy8GgWe6/
 file saved:  ./dataset/img/경복궁_BqRpy8GgWe6.jpg
 Okay

[  300  ]
  https://www.instagram.com/p/BqQBWk_AZ1Q/
 file saved:  ./dataset/img/

 file saved:  ./dataset/img/경복궁_BqQu2u9jg2m.jpg
 Okay

[  368  ]
  https://www.instagram.com/p/BqRbbrYnrk3/
 file saved:  ./dataset/img/경복궁_BqRbbrYnrk3.jpg
 Okay

[  369  ]
  https://www.instagram.com/p/BqQyaPnjJWY/
 file saved:  ./dataset/img/경복궁_BqQyaPnjJWY.jpg
 Okay

[  370  ]
  https://www.instagram.com/p/BqRIotiDUVJ/
 file saved:  ./dataset/img/경복궁_BqRIotiDUVJ.jpg
 Okay

[  371  ]
  https://www.instagram.com/p/BqQuOQyBNGV/
 file saved:  ./dataset/img/경복궁_BqQuOQyBNGV.jpg
 Okay

[  372  ]
  https://www.instagram.com/p/BqQt7slBLnv/
 file saved:  ./dataset/img/경복궁_BqQt7slBLnv.jpg
 Okay

[  373  ]
  https://www.instagram.com/p/BqRc9-VBGkZ/
 file saved:  ./dataset/img/경복궁_BqRc9-VBGkZ.jpg
 Okay

[  374  ]
  https://www.instagram.com/p/BqRLC82g0C1/
 file saved:  ./dataset/img/경복궁_BqRLC82g0C1.jpg
 Okay

[  375  ]
  https://www.instagram.com/p/BqQz3HxAIEl/
 file saved:  ./dataset/img/경복궁_BqQz3HxAIEl.jpg
 Okay

[  376  ]
  https://www.instagram.com/p/BqRf-VwhN08/
 file saved:  ./dataset/img/

 file saved:  ./dataset/img/경복궁_BqRQP6CgIX2.jpg
 Okay

[  444  ]
  https://www.instagram.com/p/BqQhEhMAcre/
 file saved:  ./dataset/img/경복궁_BqQhEhMAcre.jpg
 Okay

[  445  ]
  https://www.instagram.com/p/BqRIvX9glKM/
 file saved:  ./dataset/img/경복궁_BqRIvX9glKM.jpg
 Okay

[  446  ]
  https://www.instagram.com/p/BqReII8gffV/
 file saved:  ./dataset/img/경복궁_BqReII8gffV.jpg
 Okay

[  447  ]
  https://www.instagram.com/p/BqRXeNLlGEr/
 file saved:  ./dataset/img/경복궁_BqRXeNLlGEr.jpg
 Okay

[  448  ]
  https://www.instagram.com/p/BqRJVSHljjV/
 file saved:  ./dataset/img/경복궁_BqRJVSHljjV.jpg
 Okay

[  449  ]
  https://www.instagram.com/p/BqRtKOrn5ZO/
 file saved:  ./dataset/img/경복궁_BqRtKOrn5ZO.jpg
 Okay

[  450  ]
  https://www.instagram.com/p/BpI6P2Pg5xQ/
 file saved:  ./dataset/img/경복궁_BpI6P2Pg5xQ.jpg
 Okay

[  451  ]
  https://www.instagram.com/p/BqRbV42B9ee/
 file saved:  ./dataset/img/경복궁_BqRbV42B9ee.jpg
 Okay

[  452  ]
  https://www.instagram.com/p/BqQ9_9_hRKi/
 file saved:  ./dataset/img/